In [1]:
#Installs
!pip install transformers
!pip install transformers[sentencepiece] sacrebleu
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.3 MB/s eta 0:00:00


In [2]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sentence_transformers import SentenceTransformer, util
import torch
from sacrebleu.metrics import CHRF
from nltk.translate.meteor_score import meteor_score
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams

In [6]:
#Load Model
pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to("cuda")
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='hin_Deva', max_length = 400)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
#Load Dataset
df=pd.read_csv('./Bhagwad_Gita.csv')
hindi_sentences=df['HinMeaning'].values
english_sentences=df['EngMeaning'].values

In [9]:
#Translation
translatedEngHin = []

for (index, (eng, hin)) in enumerate(zip(english_sentences, hindi_sentences)):
    print(index+1)
    print("English:", eng)
    prediction = translator(eng)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", hin)
    print("------------------------------")
    translatedEngHin.append(translation)

1
English: 1.1 Dhritarashtra said  What did my people and the sons of Pandu do when they had assembled
together eager for battle on the holy plain of Kurukshetra, O Sanjaya.
Translated:  1.1 धृतराष्ट्र ने कहा कि मेरे लोगों और पांडु के पुत्रों ने क्या किया जब वे कुरूक्षक्षेत्र के पवित्र मैदान पर युद्ध के लिए इकट्ठे हुए थे, हे संजय।
Original:  ।।1.1।।धृतराष्ट्र ने कहा -- हे संजय ! धर्मभूमि कुरुक्षेत्र में एकत्र हुए युद्ध के इच्छुक (युयुत्सव:) मेरे और पाण्डु के पुत्रों ने क्या किया?
------------------------------
2
English: 1.2. Sanjaya said  Having seen the army of the Pandavas drawn up in battle-array,
King Duryodhana then approached his teacher (Drona) and spoke these words.
Translated:  संजय ने कहा कि युद्ध-शैली में पंडवों की सेना को तैयार करते हुए राजा दुर्योधन ने अपने गुरु (ड्रोना) के पास जाकर ये शब्द बोले।
Original:  ।।1.2।।संजय ने कहा -- पाण्डव-सैन्य की व्यूह रचना देखकर राजा दुर्योधन ने आचार्य द्रोण के पास जाकर ये वचन कहे।
------------------------------
3
English: 1.3. "Behold, O 

In [10]:
#Saving the results
comaparision = pd.DataFrame({'Original': hindi_sentences, 'Translated': translatedEngHin})
comaparision.to_csv('/content/BG-NLLB200-Translated-Hin_Eng.csv', index=False)

###Calculating the various accuracy scores
---
```
*   BLEU: 41.77
*   CHRF: 39.1608
*   STS:  0.6788
*   METEOR: 0.3195
*   PERPLEXITY:
```

In [11]:
#BLEU Score
references = [[hindi] for hindi in hindi_sentences]  # Create reference list of lists
candidates = translatedEngHin  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.3971


In [12]:
# STS Score
# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(hindi_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedEngHin, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STS score: 0.5257


In [13]:
# CHRF
# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedEngHin, [[hindi] for hindi in hindi_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")

CHRF score: 46.7076


In [14]:
# METEOR Score
# Download required NLTK data (if not already downloaded)
nltk.download('punkt')  # Download the 'punkt' resource for sentence tokenization
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
hindi_sentences_tokenized = [nltk.word_tokenize(sent) for sent in hindi_sentences]
translatedEngHin_tokenized = [nltk.word_tokenize(sent) for sent in translatedEngHin]

# Calculate METEOR score
from nltk.translate.meteor_score import meteor_score # Import the meteor_score function
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(hindi_sentences_tokenized, translatedEngHin_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


METEOR score: 0.2321


In [15]:
# PERPLEXITY
def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
original_model = train_model(hindi_sentences)
translation_model = train_model(translatedEngHin)

# Calculate perplexities
original_perplexity = calculate_perplexity(original_model, hindi_sentences)
translated_perplexity = calculate_perplexity(translation_model, translatedEngHin)

print(f"Hindi Perplexity: {original_perplexity:.4f}")
print(f"Translated hindi Perplexity: {translated_perplexity:.4f}")

Hindi Perplexity: 17.8535
Translated hindi Perplexity: 18.3004
